In [3]:
path = "~/calpoly/Data301/Data301FinalProject/datasets/processed/201901_final.csv"

In [4]:
import pandas as pd
import numpy as np

ks201901_02 = pd.read_csv(path)
ks201901_02.head()


,id,backers_count,blurb,country,created_at,currency,deadline,goal,launched_at,name,...,usd_pledged,parent_category,category,created_at_clean,state_changed_at_clean,launched_at_clean,deadline_clean,project_duration,time_til_state_changed,avg_pledge
0,1504859185,1,An eco-friendly coffee table that is both func...,US,1473631551,USD,1480966943,5000.0,1478371343,Industrial Bamboo Table,...,240.000000,crafts,woodworking,2016-09-11 22:05:51,2016-12-05 19:42:23,2016-11-05 18:42:23,2016-12-05 19:42:23,7335392,7335392,240.000000
1,49266114,3,We take digital uploads and make them handpain...,CA,1436540372,CAD,1440417634,1000.0,1437480034,"Custom Pet Portraits on Canvas- ""From Pixels t...",...,247.950175,art,painting,2015-07-10 14:59:32,2015-08-24 12:00:34,2015-07-21 12:00:34,2015-08-24 12:00:34,3877262,3877262,82.650058
2,1228074690,243,We are a team of restaurant pros looking to fu...,US,1427218874,USD,1431706954,35000.0,1429114954,The Barmobile: Boston's Mobile Cocktail Cateri...,...,41738.000000,food,food trucks,2015-03-24 17:41:14,2015-05-15 16:22:34,2015-04-15 16:22:34,2015-05-15 16:22:34,4488080,4488080,171.761317
3,330962986,27,"Loosely-based on a Lakota legend, Grandfather ...",US,1495110632,USD,1500217383,3000.0,1496329383,Grandfather Thunder & The Night Horses,...,3115.000000,publishing,children's books,2017-05-18 12:30:32,2017-07-16 15:03:04,2017-06-01 15:03:03,2017-07-16 15:03:03,5106751,5106752,115.370370
4,1657821447,3,Save me is a feature film about a depression s...,IE,1450121921,EUR,1455328590,15000.0,1450144590,Save Me-A film to hightlight depression (Cance...,...,660.680598,film & video,movie theaters,2015-12-14 19:38:41,2016-02-10 00:54:26,2015-12-15 01:56:30,2016-02-13 01:56:30,5206669,4943745,220.226866


In [5]:
len(ks201901_02)

180934

In [7]:
ks201901_02["state"].value_counts()

successful    94425
failed        74469
canceled       8681
live           2736
suspended       623
Name: state, dtype: int64

Do this one like we have been doing for the other model in machinelearning and in moremachinelearning